rm -rf *.csv

In [1]:
import numpy as np
import pandas as pd
import time
import csv
import random
import matplotlib.pyplot as plt
from PIL import Image
from barbar import Bar

import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split

import sklearn.metrics as metrics
from sklearn.metrics import roc_auc_score

use_gpu = torch.cuda.is_available()
print(use_gpu)

train_path = '/gpfs/data/denizlab/Datasets/Public/CheXpert-v1.0/train.csv'
valid_path = '/gpfs/data/denizlab/Datasets/Public/CheXpert-v1.0/valid.csv'


Traindata = pd.read_csv(train_path)
Traindata = Traindata[Traindata['Path'].str.contains("frontal")] # use only frontal images
#Traindata = Traindata[500:]
Traindata.to_csv('/gpfs/data/denizlab/Users/skr2369/Chexpert/CheXpert-v1/U1-V1/train_mod.csv', index = False)
print("Train data length:", len(Traindata))

Validdata = pd.read_csv(valid_path)
Validdata = Validdata[Validdata['Path'].str.contains("frontal")] # use only frontal images
Validdata.to_csv('/gpfs/data/denizlab/Users/skr2369/Chexpert/CheXpert-v1/U1-V1/valid_mod.csv', index = False)
print("Valid data length:", len(Validdata))

# Testdata = Traindata.head(500) # use first 500 training data as test data (obs ratio is almost same!)
# Testdata.to_csv('/gpfs/data/denizlab/Users/skr2369/Chexpert/CheXpert-v1/test_mod.csv', index = False)
# print("Test data length:", len(Testdata))

pathFileTrain = '/gpfs/data/denizlab/Users/skr2369/Chexpert/CheXpert-v1/U1-V1/train_mod.csv'
pathFileValid = '/gpfs/data/denizlab/Users/skr2369/Chexpert/CheXpert-v1/U1-V1/valid_mod.csv'
# pathFileTest = '/gpfs/data/denizlab/Users/skr2369/Chexpert/CheXpert-v1/test_mod.csv'

# Neural network parameters:
nnIsTrained = False     # pre-trained using ImageNet
nnClassCount = 14       # dimension of the output

# Training settings: batch size, maximum number of epochs
trBatchSize = 16
trMaxEpoch = 3

# Parameters related to image transforms: size of the down-scaled image, cropped image
imgtransResize = (320, 320)
imgtransCrop = 224

# Class names
class_names = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 
               'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 
               'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices']

class CheXpertDataSet(Dataset):
    def __init__(self, data_PATH, transform = None, policy = "ones"):
        """
        data_PATH: path to the file containing images with corresponding labels.
        transform: optional transform to be applied on a sample.
        Upolicy: name the policy with regard to the uncertain labels.
        """
        image_names = []
        labels = []

        with open(data_PATH, "r") as f:
            csvReader = csv.reader(f)
            next(csvReader, None) # skip the header
            for line in csvReader:
                image_name = line[0]
                label = line[5:]
                
                for i in range(14):
                    if label[i]:
                        a = float(label[i])
                        if a == 1:
                            label[i] = 1
                        elif a == -1:
                            if policy == "ones":
                                label[i] = 1
                            elif policy == "zeroes":
                                label[i] = 0
                            else:
                                label[i] = 0
                        else:
                            label[i] = 0
                    else:
                        label[i] = 0
                
                image_names.append('/gpfs/data/denizlab/Datasets/Public/' + image_name)
        
#                 image_names.append('./' + image_name)
                labels.append(label)

        self.image_names = image_names
        self.labels = labels
        self.transform = transform

    def __getitem__(self, index):
        """Take the index of item and returns the image and its labels"""
        image_name = self.image_names[index]
        image = Image.open(image_name).convert('RGB')
        label = self.labels[index]
        if self.transform is not None:
            image = self.transform(image)
        return image, torch.FloatTensor(label)

    def __len__(self):
        return len(self.image_names)
    

IMAGENET_MEAN = [0.485, 0.456, 0.406]  # mean of ImageNet dataset(for normalization)
IMAGENET_STD = [0.229, 0.224, 0.225]   # std of ImageNet dataset(for normalization)

# Tranform data
normalize = transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
transformList = []

transformList.append(transforms.Resize((imgtransCrop, imgtransCrop))) # 224
# transformList.append(transforms.RandomResizedCrop(imgtransCrop))
# transformList.append(transforms.RandomHorizontalFlip())
transformList.append(transforms.ToTensor())
# transformList.append(normalize)
transformSequence = transforms.Compose(transformList)

# Load dataset
datasetTrain = CheXpertDataSet(pathFileTrain, transformSequence, policy = "ones")
print("Train data length:", len(datasetTrain))
dataLoaderTrain = DataLoader(dataset=datasetTrain, batch_size=trBatchSize, shuffle=True,  num_workers=20, pin_memory=True)


datasetValid = CheXpertDataSet(pathFileValid, transformSequence)
print("Valid data length:", len(datasetValid))
dataLoaderVal = DataLoader(dataset = datasetValid, batch_size = trBatchSize, 
                           shuffle = False, num_workers = 2, pin_memory = True)

# data("Test data length:", len(datasetTest))


class CheXpertTrainer():

    def train(model, dataLoaderTrain, dataLoaderVal, nnClassCount, trMaxEpoch, checkpoint):
        optimizer = optim.Adam(model.parameters(), lr = 0.0001, # setting optimizer & scheduler
                               betas = (0.9, 0.999), eps = 1e-08, weight_decay = 0) 
        loss = torch.nn.BCELoss() # setting loss function
        
        if checkpoint != None and use_gpu: # loading checkpoint
            modelCheckpoint = torch.load(checkpoint)
            model.load_state_dict(modelCheckpoint['state_dict'])
            optimizer.load_state_dict(modelCheckpoint['optimizer'])
            
        # Train the network
        lossMIN = 100000
        train_start = []
        train_end = []
        for epochID in range(0, trMaxEpoch):
            train_start.append(time.time()) # training starts
            losst = CheXpertTrainer.epochTrain(model, dataLoaderTrain, optimizer, trMaxEpoch, nnClassCount, loss)
            train_end.append(time.time()) # training ends
#             lossv = CheXpertTrainer.epochVal(model, dataLoaderVal, optimizer, trMaxEpoch, nnClassCount, loss)
            print("Training loss: {:.3f},".format(losst))#, "Valid loss: {:.3f}".format(lossv))
            
            if losst < lossMIN:
                lossMIN = losst
                torch.save({'epoch': epochID + 1, 'state_dict': model.state_dict(), 
                            'best_loss': lossMIN, 'optimizer' : optimizer.state_dict()}, 
                           'm-epoch_FL' + str(epochID + 1) + '.pth.tar')
                print('Epoch ' + str(epochID + 1) + ' [save] loss = ' + str(losst))
            else:
                print('Epoch ' + str(epochID + 1) + ' [----] loss = ' + str(losst))

        train_time = np.array(train_end) - np.array(train_start)
        print("Training time for each epoch: {} seconds".format(train_time.round(0)))
        params = model.state_dict()
        return params
       
        
    def epochTrain(model, dataLoaderTrain, optimizer, epochMax, classCount, loss):
        losstrain = 0
        model.train()

        for batchID, (varInput, target) in enumerate(Bar(dataLoaderTrain)):
            
            varTarget = target.cuda(non_blocking = True)
            varOutput = model(varInput)
            lossvalue = loss(varOutput, varTarget)
                       
            optimizer.zero_grad()
            lossvalue.backward()
            optimizer.step()
            
            losstrain += lossvalue.item()
            
        return losstrain / len(dataLoaderTrain)

    
    def computeAUROC(dataGT, dataPRED, classCount):
        # Computes area under ROC curve 
        # dataGT: ground truth data
        # dataPRED: predicted data
        outAUROC = []
        datanpGT = dataGT.cpu().numpy()
        datanpPRED = dataPRED.cpu().numpy()
        
        for i in range(classCount):
            try:
                outAUROC.append(roc_auc_score(datanpGT[:, i], datanpPRED[:, i]))
            except ValueError:
                pass
        return outAUROC
    
    
    def test(model, dataLoaderTest, nnClassCount, checkpoint, class_names):
        cudnn.benchmark = True
        
        if checkpoint != None and use_gpu:
            modelCheckpoint = torch.load(checkpoint)
            model.load_state_dict(modelCheckpoint['state_dict'])

        if use_gpu:
            outGT = torch.FloatTensor().cuda()
            outPRED = torch.FloatTensor().cuda()
        else:
            outGT = torch.FloatTensor()
            outPRED = torch.FloatTensor()
       
        model.eval()
        
        with torch.no_grad():
            for i, (input, target) in enumerate(dataLoaderTest):

                target = target.cuda()
                outGT = torch.cat((outGT, target), 0).cuda()

                bs, c, h, w = input.size()
                varInput = input.view(-1, c, h, w)
            
                out = model(varInput)
                outPRED = torch.cat((outPRED, out), 0)
        aurocIndividual = CheXpertTrainer.computeAUROC(outGT, outPRED, nnClassCount)
        aurocMean = np.array(aurocIndividual).mean()
        print('AUROC mean ', aurocMean)
        
        for i in range (0, len(aurocIndividual)):
            print(class_names[i], ' ', aurocIndividual[i])
        
        return outGT, outPRED

True
Train data length: 191027
Valid data length: 202
Train data length: 191027
Valid data length: 202


In [2]:
class DenseNet121(nn.Module):
    """Model modified.
    The architecture of our model is the same as standard DenseNet121
    except the classifier layer which has an additional sigmoid function.
    """
    def __init__(self, out_size):
        super(DenseNet121, self).__init__()
        self.densenet121 = torchvision.models.densenet121(pretrained = False)
        num_ftrs = self.densenet121.classifier.in_features
        self.densenet121.classifier = nn.Sequential(
            nn.Linear(num_ftrs, out_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.densenet121(x)
        return x

In [3]:
# densenet model
class densenet_model(nn.Module):
    def __init__(self, size, features_dim, out_size, pretrained=False):
        super(densenet_model, self).__init__()
        
        if size==121:
            self.backbone = torchvision.models.densenet121(pretrained = False)
        else:
            raise NotImplementedError(f"ResNet with size {size} is not implemented!")

        #self.backbone.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.feature_dim_in = self.backbone.classifier.in_features
        self.backbone.classifier = nn.Linear(in_features=self.feature_dim_in, out_features=features_dim, bias=True)
        self.classifier = nn.Sequential(
            nn.Linear(features_dim, out_size),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [19]:
# resnet model
class resnet_model(nn.Module):
    def __init__(self, size, features_dim, out_size, pretrained=False):
        super(resnet_model, self).__init__()
        
        if size==18:
            self.backbone = torchvision.models.resnet18(pretrained=pretrained)
        elif size==50:
            self.backbone = torchvision.models.resnet50(pretrained=pretrained)
        elif size==101:
            self.backbone = torchvision.models.resnet101(pretrained=pretrained)
        else:
            raise NotImplementedError(f"ResNet with size {size} is not implemented!")

        #self.backbone.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.feature_dim_in = self.backbone.fc.weight.shape[1]
        #self.backbone.fc = nn.Linear(in_features=self.feature_dim_in, out_features=features_dim, bias=True)
        self.backbone.fc = nn.Identity()
        self.classifier = nn.Sequential(
            nn.Linear(features_dim, out_size),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [5]:
# vit model
class vit_model(nn.Module):
    def __init__(self, size, features_dim, out_size, pretrained=False, freeze_pos_embed=False, **kwargs):
        super(vit_model, self).__init__()
        
        if freeze_pos_embed:
            pass
        else:
            if size=="base":
                model_kwargs = dict(
                    patch_size=16, embed_dim=768, depth=12, num_heads=12, num_classes=0, **kwargs
                )
                self.backbone = _create_vision_transformer("vit_base_patch16_224", pretrained=pretrained, **model_kwargs)
            else:
                pass

        self.classifier = nn.Sequential(
            nn.Linear(features_dim, out_size),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [20]:
size = 50
features_dim = 2048
out_size = 14

model = resnet_model(size, features_dim, out_size, pretrained=True).cuda() # Step 0: Initialize global model and load the model
model = torch.nn.DataParallel(model).cuda()
#model = model.cuda()

In [7]:
from timm.models.vision_transformer import _create_vision_transformer
from timm.models.layers import PatchEmbed
from timm.models.vision_transformer import VisionTransformer, _cfg

/gpfs/share/skynet/apps/anaconda3/envs/opence_env_1.5.1/lib/python3.8/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)


In [8]:
size = "base"
features_dim = 768
out_size = nnClassCount
model = vit_model(size, features_dim, out_size, pretrained=True).cuda() # Step 0: Initialize global model and load the m
model = torch.nn.DataParallel(model).cuda()

In [6]:
# model = DenseNet121(nnClassCount).cuda() # Step 0: Initialize global model and load the model
# model = torch.nn.DataParallel(model).cuda()

In [9]:
ls

DENSENET_SIMCLR.err       VIT_CLIP.out         m-epoch_FL2.pth.tar
DENSENET_SIMCLR.out       VIT_SIMCLR.err       m-epoch_FL3.pth.tar
DENSENET_SLIP_SIMCLR.err  VIT_SIMCLR.out       m-epoch_FL_ResNet1.pth.tar
DENSENET_SLIP_SIMCLR.out  VIT_SLIP_VICREG.err  m-epoch_FL_ResNet2.pth.tar
DENSENET_SLIP_VICREG.err  VIT_SLIP_VICREG.out  m-epoch_FL_ResNet3.pth.tar
DENSENET_SLIP_VICREG.out  VIT_VICREG.err       model_saved/
DENSENET_VICREG.err       VIT_VICREG.out       submitit.bsub
DENSENET_VICREG.out       __pycache__/         train_mod.csv
Performance_Check.ipynb   base3.py             valid_mod.csv
VIT_CLIP.err              m-epoch_FL1.pth.tar


In [21]:
def load_model(load_path, model):
    load_path = load_path
    checkpoint = torch.load(load_path)
    state_dict = {k.replace("img_backbone.", "module."): v for k, v in checkpoint['state_dict'].items()}
    model.load_state_dict(state_dict, strict=False)

# ResNet-50

## Original

In [53]:
path = "model_saved/m-epoch_FL_resnet1.pth.tar"
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.8104620559715634
No Finding   0.9134615384615385
Enlarged Cardiomediastinum   0.5973490427098674
Cardiomegaly   0.8078208556149733
Lung Opacity   0.9039718451483157
Lung Lesion   0.2587064676616916
Edema   0.9309523809523809
Consolidation   0.8549632352941177
Pneumonia   0.8537371134020617
Atelectasis   0.8056692913385827
Pneumothorax   0.8996336996336997
Pleural Effusion   0.9102128623188406
Pleural Other   0.8557213930348259
Fracture   0.9438070020594292


In [54]:
path1 = 'model_saved/m-epoch_FL_resnet2.pth.tar'
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path1, class_names)

AUROC mean  0.8031102929581553
No Finding   0.8841783216783217
Enlarged Cardiomediastinum   0.6835542464408444
Cardiomegaly   0.82174688057041
Lung Opacity   0.889894419306184
Lung Lesion   0.5174129353233831
Edema   0.906547619047619
Consolidation   0.85625
Pneumonia   0.7190721649484536
Atelectasis   0.8473490813648294
Pneumothorax   0.882051282051282
Pleural Effusion   0.9254981884057971
Pleural Other   0.5621890547263682
Fracture   0.9446896145925272


In [55]:
path3 = 'model_saved/m-epoch_FL_resnet3.pth.tar'
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path3, class_names)

AUROC mean  0.7645676928084052
No Finding   0.8948863636363638
Enlarged Cardiomediastinum   0.4993618065783014
Cardiomegaly   0.7725044563279857
Lung Opacity   0.8950226244343892
Lung Lesion   0.3532338308457711
Edema   0.9397321428571428
Consolidation   0.7876838235294117
Pneumonia   0.720360824742268
Atelectasis   0.750236220472441
Pneumothorax   0.9003663003663004
Pleural Effusion   0.9199501811594204
Pleural Other   0.572139303482587
Fracture   0.9339021280768854


## CLIP

In [64]:
load_path = "model_saved/m-epoch_FL_resnet_clip_lp1.pth.tar"
load_model(load_path, model)

In [65]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.6313265872784167
No Finding   0.6579982517482517
Enlarged Cardiomediastinum   0.4764850270004909
Cardiomegaly   0.6485071301247772
Lung Opacity   0.6543991955756662
Lung Lesion   0.37810945273631846
Edema   0.5888392857142857
Consolidation   0.6878676470588235
Pneumonia   0.630798969072165
Atelectasis   0.6629921259842521
Pneumothorax   0.7553113553113553
Pleural Effusion   0.6661005434782609
Pleural Other   0.8308457711442786
Fracture   0.5689908796704913


In [66]:
load_path = "model_saved/m-epoch_FL_resnet_clip_lp2.pth.tar"
load_model(load_path, model)

In [67]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.6627435475137265
No Finding   0.6848776223776223
Enlarged Cardiomediastinum   0.45645557191948943
Cardiomegaly   0.6723484848484849
Lung Opacity   0.6712921065862243
Lung Lesion   0.5472636815920398
Edema   0.6141369047619047
Consolidation   0.7251838235294117
Pneumonia   0.6417525773195876
Atelectasis   0.6873490813648293
Pneumothorax   0.7216117216117216
Pleural Effusion   0.6873867753623188
Pleural Other   0.9303482587064676
Fracture   0.5756595076983426


In [68]:
load_path = "model_saved/m-epoch_FL_resnet_clip_lp3.pth.tar"
load_model(load_path, model)

In [69]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.6620178315650629
No Finding   0.7200611888111889
Enlarged Cardiomediastinum   0.4436917034855179
Cardiomegaly   0.6926247771836007
Lung Opacity   0.6779286073403721
Lung Lesion   0.592039800995025
Edema   0.6188988095238095
Consolidation   0.7136029411764706
Pneumonia   0.597938144329897
Atelectasis   0.6798950131233595
Pneumothorax   0.6564102564102564
Pleural Effusion   0.7002943840579711
Pleural Other   0.9353233830845771
Fracture   0.5775228008237717


## VICREG

In [70]:
load_path = "model_saved/m-epoch_FL_resnet_vicreg_lp1.pth.tar"
load_model(load_path, model)

In [71]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7476982794136359
No Finding   0.920236013986014
Enlarged Cardiomediastinum   0.6136475208640157
Cardiomegaly   0.7240418894830659
Lung Opacity   0.8330819507290095
Lung Lesion   0.0
Edema   0.8938988095238095
Consolidation   0.859375
Pneumonia   0.838917525773196
Atelectasis   0.7845669291338583
Pneumothorax   0.854945054945055
Pleural Effusion   0.8343523550724637
Pleural Other   0.8507462686567164
Fracture   0.7122683142100619


In [72]:
load_path = "model_saved/m-epoch_FL_resnet_vicreg_lp2.pth.tar"
load_model(load_path, model)

In [73]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7434389552364055
No Finding   0.9213286713286714
Enlarged Cardiomediastinum   0.578105056455572
Cardiomegaly   0.7302807486631017
Lung Opacity   0.8398190045248869
Lung Lesion   0.0
Edema   0.8999999999999999
Consolidation   0.8632352941176471
Pneumonia   0.82409793814433
Atelectasis   0.791496062992126
Pneumothorax   0.8249084249084248
Pleural Effusion   0.8367300724637682
Pleural Other   0.845771144278607
Fracture   0.7089340001961362


In [74]:
load_path = "model_saved/m-epoch_FL_resnet_vicreg_lp3.pth.tar"
load_model(load_path, model)

In [75]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.741722452848864
No Finding   0.9191433566433567
Enlarged Cardiomediastinum   0.5983308787432499
Cardiomegaly   0.7274955436720142
Lung Opacity   0.8510809451985922
Lung Lesion   0.0
Edema   0.9007440476190476
Consolidation   0.8595588235294118
Pneumonia   0.8260309278350516
Atelectasis   0.791286089238845
Pneumothorax   0.7985347985347985
Pleural Effusion   0.846580615942029
Pleural Other   0.8109452736318408
Fracture   0.7126605864469943


## SIMCLR

In [76]:
load_path = "model_saved/m-epoch_FL_resnet_simclr_lp1.pth.tar"
load_model(load_path, model)

In [77]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7426349191011945
No Finding   0.9036276223776224
Enlarged Cardiomediastinum   0.5560137457044674
Cardiomegaly   0.7540106951871658
Lung Opacity   0.844746103569633
Lung Lesion   0.00995024875621886
Edema   0.9081845238095237
Consolidation   0.8448529411764706
Pneumonia   0.7789948453608246
Atelectasis   0.7870866141732284
Pneumothorax   0.8695970695970695
Pleural Effusion   0.8594882246376812
Pleural Other   0.8159203980099503
Fracture   0.7217809159556733


In [78]:
load_path = "model_saved/m-epoch_FL_resnet_simclr_lp2.pth.tar"
load_model(load_path, model)

In [79]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7424789327341933
No Finding   0.8977272727272727
Enlarged Cardiomediastinum   0.5472754050073638
Cardiomegaly   0.7663770053475936
Lung Opacity   0.8582202111613876
Lung Lesion   0.00995024875621886
Edema   0.9107142857142857
Consolidation   0.8511029411764706
Pneumonia   0.7725515463917525
Atelectasis   0.7886614173228345
Pneumothorax   0.8600732600732601
Pleural Effusion   0.8633378623188406
Pleural Other   0.7960199004975125
Fracture   0.7302147690497205


In [80]:
load_path = "model_saved/m-epoch_FL_resnet_simclr_lp3.pth.tar"
load_model(load_path, model)

In [81]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7420404182204514
No Finding   0.9014423076923077
Enlarged Cardiomediastinum   0.5640648011782032
Cardiomegaly   0.7584670231729055
Lung Opacity   0.8564102564102565
Lung Lesion   0.00497512437810943
Edema   0.9138392857142856
Consolidation   0.8544117647058824
Pneumonia   0.776417525773196
Atelectasis   0.7844619422572179
Pneumothorax   0.8454212454212454
Pleural Effusion   0.8677536231884058
Pleural Other   0.7810945273631841
Fracture   0.7377660096106698


## SLIP VICREG

In [82]:
load_path = "model_saved/m-epoch_FL_resnet_slip_vicreg_lp1.pth.tar"
load_model(load_path, model)

In [83]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7264217392332301
No Finding   0.9134615384615383
Enlarged Cardiomediastinum   0.5010309278350515
Cardiomegaly   0.7697192513368984
Lung Opacity   0.8518853695324283
Lung Lesion   0.00995024875621886
Edema   0.8660714285714285
Consolidation   0.8485294117647059
Pneumonia   0.8234536082474228
Atelectasis   0.8048293963254594
Pneumothorax   0.7860805860805861
Pleural Effusion   0.8290307971014492
Pleural Other   0.7412935323383085
Fracture   0.6981465136804943


In [84]:
load_path = "model_saved/m-epoch_FL_resnet_slip_vicreg_lp2.pth.tar"
load_model(load_path, model)

In [85]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7327521256297762
No Finding   0.9138986013986015
Enlarged Cardiomediastinum   0.5478645066273932
Cardiomegaly   0.7723930481283422
Lung Opacity   0.8570135746606334
Lung Lesion   0.00995024875621886
Edema   0.8755952380952381
Consolidation   0.8568014705882353
Pneumonia   0.8234536082474226
Atelectasis   0.8032545931758531
Pneumothorax   0.7897435897435897
Pleural Effusion   0.8387681159420289
Pleural Other   0.7313432835820896
Fracture   0.7056977542414435


In [86]:
load_path = "model_saved/m-epoch_FL_resnet_slip_vicreg_lp3.pth.tar"
load_model(load_path, model)

In [87]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7372218977129171
No Finding   0.913243006993007
Enlarged Cardiomediastinum   0.5701521845851742
Cardiomegaly   0.7691622103386809
Lung Opacity   0.8597285067873303
Lung Lesion   0.00995024875621886
Edema   0.8846726190476191
Consolidation   0.859375
Pneumonia   0.8202319587628866
Atelectasis   0.8026246719160105
Pneumothorax   0.8014652014652015
Pleural Effusion   0.8427309782608696
Pleural Other   0.736318407960199
Fracture   0.714229675394724


## SLIP SIMCLR

In [88]:
load_path = "model_saved/m-epoch_FL_resnet_slip_simclr_lp1.pth.tar"
load_model(load_path, model)

In [89]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7495057706267672
No Finding   0.9040646853146853
Enlarged Cardiomediastinum   0.5833087874324988
Cardiomegaly   0.7719474153297683
Lung Opacity   0.8530920060331825
Lung Lesion   0.07462686567164178
Edema   0.8976190476190476
Consolidation   0.8476102941176471
Pneumonia   0.8408505154639174
Atelectasis   0.7939107611548557
Pneumothorax   0.8373626373626373
Pleural Effusion   0.8471467391304348
Pleural Other   0.791044776119403
Fracture   0.7009904873982544


In [90]:
load_path = "model_saved/m-epoch_FL_resnet_slip_simclr_lp2.pth.tar"
load_model(load_path, model)

In [91]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7536501111799981
No Finding   0.9079982517482517
Enlarged Cardiomediastinum   0.5868433971526755
Cardiomegaly   0.7692736185383244
Lung Opacity   0.8578179989944695
Lung Lesion   0.05472636815920395
Edema   0.9074404761904762
Consolidation   0.8523897058823531
Pneumonia   0.821520618556701
Atelectasis   0.7956955380577427
Pneumothorax   0.8454212454212454
Pleural Effusion   0.8519021739130435
Pleural Other   0.835820895522388
Fracture   0.7106011572030989


In [92]:
load_path = "model_saved/m-epoch_FL_resnet_slip_simclr_lp3.pth.tar"
load_model(load_path, model)

In [93]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7533132684209467
No Finding   0.9130244755244755
Enlarged Cardiomediastinum   0.5886107020127639
Cardiomegaly   0.7604723707664883
Lung Opacity   0.8571141277023631
Lung Lesion   0.04477611940298509
Edema   0.9089285714285714
Consolidation   0.8591911764705883
Pneumonia   0.8131443298969073
Atelectasis   0.789501312335958
Pneumothorax   0.8358974358974358
Pleural Effusion   0.8588088768115942
Pleural Other   0.8507462686567164
Fracture   0.7128567225654605


# DENSENET

In [46]:
size = 121
features_dim = 2048
out_size = 14

model = densenet_model(size, features_dim, out_size, pretrained=True).cuda() # Step 0: Initialize global model and load the model
model = torch.nn.DataParallel(model).cuda()
#model = model.cuda()

In [47]:
ls

Performance_Check.ipynb  m-epoch_FL2.pth.tar         m-epoch_FL_ResNet3.pth.tar
__pycache__/             m-epoch_FL3.pth.tar         model_saved/
base3.py                 m-epoch_FL_ResNet1.pth.tar  train_mod.csv
m-epoch_FL1.pth.tar      m-epoch_FL_ResNet2.pth.tar  valid_mod.csv


In [48]:
def load_model(load_path, model):
    load_path = load_path
    checkpoint = torch.load(load_path)
    state_dict = {k.replace("img_backbone.", "module."): v for k, v in checkpoint['state_dict'].items()}
    model.load_state_dict(state_dict, strict=False)

## Original

In [49]:
load_path = "model_saved/m-epoch_FL_densenet1.pth.tar"
load_model(load_path, model)

In [50]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.8228874986115555
No Finding   0.9071241258741258
Enlarged Cardiomediastinum   0.5201767304860089
Cardiomegaly   0.8358957219251337
Lung Opacity   0.9156359979889391
Lung Lesion   0.34328358208955223
Edema   0.9342261904761905
Consolidation   0.8536764705882354
Pneumonia   0.8395618556701031
Atelectasis   0.8192125984251969
Pneumothorax   0.9472527472527472
Pleural Effusion   0.9311594202898551
Pleural Other   0.9104477611940298
Fracture   0.9398842796901049


In [51]:
load_path = "model_saved/m-epoch_FL_densenet2.pth.tar"
load_model(load_path, model)

In [52]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7934229595768966
No Finding   0.9112762237762237
Enlarged Cardiomediastinum   0.4650957290132548
Cardiomegaly   0.7631461675579323
Lung Opacity   0.9166415284062344
Lung Lesion   0.29850746268656714
Edema   0.9525297619047619
Consolidation   0.8284926470588235
Pneumonia   0.714561855670103
Atelectasis   0.8035695538057743
Pneumothorax   0.9164835164835164
Pleural Effusion   0.9421422101449275
Pleural Other   0.8606965174129353
Fracture   0.9413553005786015


In [53]:
load_path = "model_saved/m-epoch_FL_densenet3.pth.tar"
load_model(load_path, model)

In [54]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.8654556482082858
No Finding   0.895104895104895
Enlarged Cardiomediastinum   0.6635247913598429
Cardiomegaly   0.827874331550802
Lung Opacity   0.902262443438914
Lung Lesion   0.7960199004975125
Edema   0.9450892857142857
Consolidation   0.8650735294117646
Pneumonia   0.7693298969072164
Atelectasis   0.8415748031496063
Pneumothorax   0.9106227106227107
Pleural Effusion   0.9279891304347826
Pleural Other   0.9601990049751243
Fracture   0.946258703540257


## VICREG

In [55]:
load_path = "model_saved/m-epoch_FL_densenet_vicreg1.pth.tar"
load_model(load_path, model)

In [56]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.773355224480067
No Finding   0.9088723776223776
Enlarged Cardiomediastinum   0.47206676485027
Cardiomegaly   0.8315508021390375
Lung Opacity   0.9243841126194068
Lung Lesion   0.07462686567164178
Edema   0.9313988095238095
Consolidation   0.8352941176470587
Pneumonia   0.7474226804123711
Atelectasis   0.8152230971128609
Pneumothorax   0.8498168498168498
Pleural Effusion   0.9210824275362319
Pleural Other   0.8159203980099503
Fracture   0.9259586152790036


In [57]:
load_path = "model_saved/m-epoch_FL_densenet_vicreg2.pth.tar"
load_model(load_path, model)

In [58]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7666066957800765
No Finding   0.8828671328671329
Enlarged Cardiomediastinum   0.3346097201767305
Cardiomegaly   0.7811942959001781
Lung Opacity   0.9139265962795374
Lung Lesion   0.14427860696517414
Edema   0.9501488095238095
Consolidation   0.8553308823529412
Pneumonia   0.7783505154639174
Atelectasis   0.7880314960629922
Pneumothorax   0.8227106227106227
Pleural Effusion   0.9367074275362319
Pleural Other   0.8606965174129353
Fracture   0.9170344218887908


In [59]:
load_path = "model_saved/m-epoch_FL_densenet_vicreg3.pth.tar"
load_model(load_path, model)

In [60]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.8320824658707987
No Finding   0.8905157342657342
Enlarged Cardiomediastinum   0.6018654884634266
Cardiomegaly   0.8540552584670231
Lung Opacity   0.8956259426847663
Lung Lesion   0.7562189054726368
Edema   0.9330357142857143
Consolidation   0.8514705882352941
Pneumonia   0.7390463917525774
Atelectasis   0.8282414698162729
Pneumothorax   0.8146520146520146
Pleural Effusion   0.9222146739130436
Pleural Other   0.8308457711442786
Fracture   0.8992841031675983


## SIMCLR

In [61]:
load_path = "model_saved/m-epoch_FL_densenet_simclr1.pth.tar"
load_model(load_path, model)

In [62]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.8091532008730893
No Finding   0.9029720279720279
Enlarged Cardiomediastinum   0.48335787923416795
Cardiomegaly   0.8595142602495544
Lung Opacity   0.9252890899949724
Lung Lesion   0.154228855721393
Edema   0.9311011904761904
Consolidation   0.8647058823529412
Pneumonia   0.8402061855670102
Atelectasis   0.826981627296588
Pneumothorax   0.9113553113553114
Pleural Effusion   0.9326313405797102
Pleural Other   0.9601990049751243
Fracture   0.9264489555751692


In [63]:
load_path = "model_saved/m-epoch_FL_densenet_simclr2.pth.tar"
load_model(load_path, model)

In [64]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.8390430767577498
No Finding   0.8909527972027972
Enlarged Cardiomediastinum   0.4679430535100638
Cardiomegaly   0.8150623885918004
Lung Opacity   0.9048768225238814
Lung Lesion   0.7164179104477613
Edema   0.9552083333333333
Consolidation   0.8400735294117647
Pneumonia   0.7525773195876289
Atelectasis   0.7959055118110236
Pneumothorax   0.9252747252747253
Pleural Effusion   0.9437273550724636
Pleural Other   0.9701492537313433
Fracture   0.9293909973521624


In [65]:
load_path = "model_saved/m-epoch_FL_densenet_simclr3.pth.tar"
load_model(load_path, model)

In [66]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.8708237216443782
No Finding   0.8813374125874126
Enlarged Cardiomediastinum   0.6653902798232696
Cardiomegaly   0.8373440285204992
Lung Opacity   0.9008547008547009
Lung Lesion   0.9701492537313433
Edema   0.933779761904762
Consolidation   0.8525735294117647
Pneumonia   0.7280927835051546
Atelectasis   0.8233070866141732
Pneumothorax   0.8717948717948718
Pleural Effusion   0.9381793478260869
Pleural Other   0.9651741293532339
Fracture   0.952731195449642


## SLIP VICREG

In [67]:
load_path = "model_saved/m-epoch_FL_densenet_slip_vicreg1.pth.tar"
load_model(load_path, model)

In [68]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7999846600703714
No Finding   0.9012237762237763
Enlarged Cardiomediastinum   0.4978890525282278
Cardiomegaly   0.838903743315508
Lung Opacity   0.9194570135746606
Lung Lesion   0.2885572139303483
Edema   0.9334821428571428
Consolidation   0.8512867647058824
Pneumonia   0.7635309278350515
Atelectasis   0.8086089238845146
Pneumothorax   0.8644688644688644
Pleural Effusion   0.917572463768116
Pleural Other   0.8756218905472637
Fracture   0.9391978032754731


In [69]:
load_path = "model_saved/m-epoch_FL_densenet_slip_vicreg2.pth.tar"
load_model(load_path, model)

In [70]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.8083247080162441
No Finding   0.8970716783216783
Enlarged Cardiomediastinum   0.5162493863524791
Cardiomegaly   0.8069295900178253
Lung Opacity   0.9165409753645047
Lung Lesion   0.2935323383084577
Edema   0.9507440476190475
Consolidation   0.8533088235294117
Pneumonia   0.7918814432989691
Atelectasis   0.7990551181102362
Pneumothorax   0.8937728937728937
Pleural Effusion   0.9276494565217391
Pleural Other   0.9154228855721394
Fracture   0.9460625674217906


In [71]:
load_path = "model_saved/m-epoch_FL_densenet_slip_vicreg3.pth.tar"
load_model(load_path, model)

In [72]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.8209272752227904
No Finding   0.8841783216783217
Enlarged Cardiomediastinum   0.6059891998036327
Cardiomegaly   0.8082664884135472
Lung Opacity   0.8830568124685771
Lung Lesion   0.6019900497512438
Edema   0.9333333333333333
Consolidation   0.8420955882352942
Pneumonia   0.7326030927835051
Atelectasis   0.8161679790026247
Pneumothorax   0.8036630036630037
Pleural Effusion   0.9291213768115942
Pleural Other   0.8756218905472637
Fracture   0.9559674414043347


## SLIP SIMCLR

In [73]:
load_path = "model_saved/m-epoch_FL_densenet_slip_simclr1.pth.tar"
load_model(load_path, model)

In [74]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.8302012123159446
No Finding   0.913243006993007
Enlarged Cardiomediastinum   0.5894943544428081
Cardiomegaly   0.8382352941176471
Lung Opacity   0.9135243841126194
Lung Lesion   0.42288557213930345
Edema   0.9297619047619048
Consolidation   0.8663602941176471
Pneumonia   0.851159793814433
Atelectasis   0.8241469816272965
Pneumothorax   0.8901098901098901
Pleural Effusion   0.9276494565217391
Pleural Other   0.8855721393034826
Fracture   0.9404726880455035


In [75]:
load_path = "model_saved/m-epoch_FL_densenet_slip_simclr2.pth.tar"
load_model(load_path, model)

In [76]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.8024862679409466
No Finding   0.9088723776223777
Enlarged Cardiomediastinum   0.5056455571919489
Cardiomegaly   0.7906639928698753
Lung Opacity   0.9152337858220211
Lung Lesion   0.20398009950248752
Edema   0.9571428571428571
Consolidation   0.8428308823529412
Pneumonia   0.7938144329896908
Atelectasis   0.7799475065616799
Pneumothorax   0.9472527472527472
Pleural Effusion   0.9358016304347826
Pleural Other   0.9253731343283582
Fracture   0.9257624791605374


In [77]:
load_path = "model_saved/m-epoch_FL_densenet_slip_simclr3.pth.tar"
load_model(load_path, model)

In [78]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.8499568525547764
No Finding   0.8981643356643356
Enlarged Cardiomediastinum   0.6579283259695631
Cardiomegaly   0.8108288770053476
Lung Opacity   0.901458019105078
Lung Lesion   0.6467661691542288
Edema   0.9369047619047619
Consolidation   0.8753676470588236
Pneumonia   0.8144329896907216
Atelectasis   0.822992125984252
Pneumothorax   0.8717948717948718
Pleural Effusion   0.936141304347826
Pleural Other   0.9203980099502488
Fracture   0.956261645582034


# Linear Probing

## Original

In [12]:
path = "model_saved/m-epoch_FL_resnet_lp1.pth.tar"
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.6419114160440883
No Finding   0.8540209790209791
Enlarged Cardiomediastinum   0.4559646539027983
Cardiomegaly   0.6678921568627452
Lung Opacity   0.808044243338361
Lung Lesion   0.04477611940298509
Edema   0.7950892857142857
Consolidation   0.7922794117647058
Pneumonia   0.6623711340206185
Atelectasis   0.7633595800524935
Pneumothorax   0.7494505494505495
Pleural Effusion   0.7778532608695653
Pleural Other   0.25373134328358204
Fracture   0.7200156908894774


In [13]:
path1 = 'model_saved/m-epoch_FL_resnet_lp2.pth.tar'
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path1, class_names)

AUROC mean  0.6551112739068264
No Finding   0.8690996503496503
Enlarged Cardiomediastinum   0.4251350024545901
Cardiomegaly   0.6705659536541889
Lung Opacity   0.804927099044746
Lung Lesion   0.02487562189054726
Edema   0.8133928571428571
Consolidation   0.7860294117647059
Pneumonia   0.7203608247422681
Atelectasis   0.7596850393700787
Pneumothorax   0.747985347985348
Pleural Effusion   0.7793251811594203
Pleural Other   0.3830845771144279
Fracture   0.7319799941159163


In [14]:
path3 = 'model_saved/m-epoch_FL_resnet_lp3.pth.tar'
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path3, class_names)

AUROC mean  0.6628867769265889
No Finding   0.8721590909090909
Enlarged Cardiomediastinum   0.4705940108001964
Cardiomegaly   0.6727941176470589
Lung Opacity   0.8143790849673203
Lung Lesion   0.02487562189054726
Edema   0.8165178571428572
Consolidation   0.8001838235294118
Pneumonia   0.731958762886598
Atelectasis   0.7649343832020997
Pneumothorax   0.7289377289377289
Pleural Effusion   0.7924592391304348
Pleural Other   0.3980099502487562
Fracture   0.7297244287535549


## CLIP

In [15]:
load_path = "model_saved/m-epoch_FL_resnet_clip_lp1.pth.tar"
load_model(load_path, model)

In [16]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.5740112820066824
No Finding   0.5563811188811189
Enlarged Cardiomediastinum   0.3696612665684831
Cardiomegaly   0.5424465240641712
Lung Opacity   0.6083459024635495
Lung Lesion   0.900497512437811
Edema   0.5626488095238096
Consolidation   0.7503676470588235
Pneumonia   0.7016752577319587
Atelectasis   0.5956955380577429
Pneumothorax   0.5655677655677656
Pleural Effusion   0.6913496376811595
Pleural Other   0.13432835820895528
Fracture   0.483181327841522


In [17]:
load_path = "model_saved/m-epoch_FL_resnet_clip_lp2.pth.tar"
load_model(load_path, model)

In [18]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.5800559281393045
No Finding   0.5749562937062936
Enlarged Cardiomediastinum   0.3858615611192931
Cardiomegaly   0.5482397504456328
Lung Opacity   0.6190045248868778
Lung Lesion   0.8855721393034826
Edema   0.5686011904761904
Consolidation   0.7255514705882353
Pneumonia   0.6804123711340206
Atelectasis   0.6121784776902885
Pneumothorax   0.5377289377289377
Pleural Effusion   0.7093523550724639
Pleural Other   0.1990049751243781
Fracture   0.4942630185348632


In [20]:
load_path = "model_saved/m-epoch_FL_resnet_clip_lp3.pth.tar"
load_model(load_path, model)

In [21]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.5760752692652003
No Finding   0.5751748251748252
Enlarged Cardiomediastinum   0.3806578301423662
Cardiomegaly   0.5007798573975044
Lung Opacity   0.6239316239316239
Lung Lesion   0.8407960199004976
Edema   0.5770833333333333
Consolidation   0.7235294117647059
Pneumonia   0.6514175257731959
Atelectasis   0.6299212598425197
Pneumothorax   0.5487179487179488
Pleural Effusion   0.6493432971014492
Pleural Other   0.2885572139303483
Fracture   0.4990683534372855


## VICREG

In [22]:
load_path = "model_saved/m-epoch_FL_resnet_vicreg_lp1.pth.tar"
load_model(load_path, model)

In [23]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7204818627489432
No Finding   0.8953234265734266
Enlarged Cardiomediastinum   0.593127147766323
Cardiomegaly   0.686608734402852
Lung Opacity   0.8393162393162393
Lung Lesion   0.05970149253731338
Edema   0.8998511904761903
Consolidation   0.8380514705882353
Pneumonia   0.7261597938144331
Atelectasis   0.7783727034120735
Pneumothorax   0.8388278388278388
Pleural Effusion   0.8125000000000001
Pleural Other   0.6716417910447761
Fracture   0.7267823869765617


In [24]:
load_path = "model_saved/m-epoch_FL_resnet_vicreg_lp2.pth.tar"
load_model(load_path, model)

In [25]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7203766446149019
No Finding   0.9034090909090909
Enlarged Cardiomediastinum   0.5671084928816887
Cardiomegaly   0.6910650623885918
Lung Opacity   0.8396178984414279
Lung Lesion   0.03482587064676612
Edema   0.9019345238095238
Consolidation   0.8352941176470587
Pneumonia   0.7564432989690721
Atelectasis   0.7722834645669292
Pneumothorax   0.8285714285714285
Pleural Effusion   0.8114809782608695
Pleural Other   0.6865671641791045
Fracture   0.7362949887221732


In [26]:
load_path = "model_saved/m-epoch_FL_resnet_vicreg_lp3.pth.tar"
load_model(load_path, model)

In [27]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7344333555610798
No Finding   0.9060314685314685
Enlarged Cardiomediastinum   0.6237604320078547
Cardiomegaly   0.7027629233511586
Lung Opacity   0.844746103569633
Lung Lesion   0.03482587064676612
Edema   0.9026785714285714
Consolidation   0.8389705882352941
Pneumonia   0.7693298969072164
Atelectasis   0.7785826771653543
Pneumothorax   0.8043956043956043
Pleural Effusion   0.8268795289855073
Pleural Other   0.7761194029850746
Fracture   0.7385505540845347


## SIMCLR

In [28]:
load_path = "model_saved/m-epoch_FL_resnet_simclr_lp1.pth.tar"
load_model(load_path, model)

In [29]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7250383930746747
No Finding   0.8988199300699301
Enlarged Cardiomediastinum   0.6108001963672066
Cardiomegaly   0.7182486631016043
Lung Opacity   0.8560080442433383
Lung Lesion   0.04477611940298509
Edema   0.9052083333333334
Consolidation   0.8628676470588235
Pneumonia   0.6920103092783505
Atelectasis   0.7733333333333333
Pneumothorax   0.8183150183150183
Pleural Effusion   0.8548460144927537
Pleural Other   0.6567164179104478
Fracture   0.7335490830636461


In [30]:
load_path = "model_saved/m-epoch_FL_resnet_simclr_lp2.pth.tar"
load_model(load_path, model)

In [31]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7257449372900044
No Finding   0.8975087412587412
Enlarged Cardiomediastinum   0.5447226313205694
Cardiomegaly   0.713903743315508
Lung Opacity   0.8543991955756661
Lung Lesion   0.03482587064676612
Edema   0.9110119047619047
Consolidation   0.8715073529411765
Pneumonia   0.7429123711340206
Atelectasis   0.7723884514435696
Pneumothorax   0.7978021978021979
Pleural Effusion   0.8624320652173912
Pleural Other   0.691542288557214
Fracture   0.7397273707953319


In [32]:
load_path = "model_saved/m-epoch_FL_resnet_simclr_lp3.pth.tar"
load_model(load_path, model)

In [33]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7350559938392067
No Finding   0.906687062937063
Enlarged Cardiomediastinum   0.6691212567501228
Cardiomegaly   0.7151292335115864
Lung Opacity   0.8518853695324284
Lung Lesion   0.04975124378109452
Edema   0.9141369047619047
Consolidation   0.8683823529411765
Pneumonia   0.7673969072164949
Atelectasis   0.7792125984251969
Pneumothorax   0.780952380952381
Pleural Effusion   0.8653759057971014
Pleural Other   0.6417910447761195
Fracture   0.7459056585270177


## SLIP VICREG

In [34]:
load_path = "model_saved/m-epoch_FL_resnet_slip_vicreg_lp1.pth.tar"
load_model(load_path, model)

In [35]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7293614136277273
No Finding   0.888986013986014
Enlarged Cardiomediastinum   0.5621993127147766
Cardiomegaly   0.749108734402852
Lung Opacity   0.8316742081447964
Lung Lesion   0.14925373134328357
Edema   0.8761904761904762
Consolidation   0.8534926470588236
Pneumonia   0.7880154639175259
Atelectasis   0.779002624671916
Pneumothorax   0.8036630036630037
Pleural Effusion   0.7963088768115942
Pleural Other   0.7114427860696517
Fracture   0.692360498185741


In [36]:
load_path = "model_saved/m-epoch_FL_resnet_slip_vicreg_lp2.pth.tar"
load_model(load_path, model)

In [37]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7320789913239374
No Finding   0.8913898601398602
Enlarged Cardiomediastinum   0.5668139420716741
Cardiomegaly   0.7391934046345812
Lung Opacity   0.8381096028154852
Lung Lesion   0.10447761194029848
Edema   0.8831845238095238
Consolidation   0.8558823529411765
Pneumonia   0.7963917525773196
Atelectasis   0.7760629921259843
Pneumothorax   0.8329670329670329
Pleural Effusion   0.8020833333333334
Pleural Other   0.7313432835820896
Fracture   0.6991271942728253


In [38]:
load_path = "model_saved/m-epoch_FL_resnet_slip_vicreg_lp3.pth.tar"
load_model(load_path, model)

In [39]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7310885846417412
No Finding   0.8933566433566433
Enlarged Cardiomediastinum   0.587039764359352
Cardiomegaly   0.7416443850267379
Lung Opacity   0.8430367018602313
Lung Lesion   0.02985074626865669
Edema   0.890327380952381
Consolidation   0.8599264705882352
Pneumonia   0.8041237113402062
Atelectasis   0.7801574803149607
Pneumothorax   0.8263736263736263
Pleural Effusion   0.8115942028985507
Pleural Other   0.736318407960199
Fracture   0.7004020790428558


## SLIP SIMCLR

In [40]:
load_path = "model_saved/m-epoch_FL_resnet_slip_simclr_lp1.pth.tar"
load_model(load_path, model)

In [41]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7158631496456963
No Finding   0.892263986013986
Enlarged Cardiomediastinum   0.5434462444771724
Cardiomegaly   0.7174688057040999
Lung Opacity   0.8522875816993464
Lung Lesion   0.01990049751243783
Edema   0.8708333333333333
Consolidation   0.8713235294117647
Pneumonia   0.8189432989690721
Atelectasis   0.7864566929133858
Pneumothorax   0.6871794871794872
Pleural Effusion   0.8144248188405797
Pleural Other   0.7412935323383085
Fracture   0.6903991370010787


In [42]:
load_path = "model_saved/m-epoch_FL_resnet_slip_simclr_lp2.pth.tar"
load_model(load_path, model)

In [43]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7197351246800687
No Finding   0.8975087412587414
Enlarged Cardiomediastinum   0.507707412862052
Cardiomegaly   0.723150623885918
Lung Opacity   0.8514831573655104
Lung Lesion   0.02487562189054726
Edema   0.8863095238095238
Consolidation   0.8737132352941177
Pneumonia   0.8060567010309279
Atelectasis   0.7828871391076115
Pneumothorax   0.7135531135531135
Pleural Effusion   0.8179347826086957
Pleural Other   0.7661691542288558
Fracture   0.705207413945278


In [44]:
load_path = "model_saved/m-epoch_FL_resnet_slip_simclr_lp3.pth.tar"
load_model(load_path, model)

In [45]:
path = None
outGT, outPRED = CheXpertTrainer.test(model, dataLoaderVal, nnClassCount, path, class_names)

AUROC mean  0.7306897074943385
No Finding   0.8964160839160839
Enlarged Cardiomediastinum   0.5680903289150712
Cardiomegaly   0.7249331550802139
Lung Opacity   0.8507792860734037
Lung Lesion   0.04477611940298509
Edema   0.8956845238095238
Consolidation   0.8783088235294119
Pneumonia   0.7880154639175259
Atelectasis   0.7783727034120735
Pneumothorax   0.7406593406593407
Pleural Effusion   0.8195199275362319
Pleural Other   0.8009950248756219
Fracture   0.7124154162989114


             Atelectasis       Cardiomegaly       Consolidation       Edema Pleural      Effusion
PAPER:  0.858(0.806,0.910)  0.832(0.773,0.890)  0.899(0.854,0.944)  0.941(0.903,0.980)  0.934(0.901,0.967)
Result:    0.85070              0.84569               0.88235              0.90595          0.92142
